In [70]:
from sqlalchemy.orm import declarative_base

# Model

In [71]:
import os
from pathlib import Path
#BASE_DIR = os.path.dirname(os.path.realpath(__file__))
BASE_DIR = Path().resolve() #esto funciona en notebook
connection_string = "sqlite:///" + os.path.join(BASE_DIR, 'site.db')
connection_string = "sqlite:///:memory:"
print (connection_string)

sqlite:///:memory:


In [72]:
from sqlalchemy import Column, Integer, String, DateTime, create_engine
from datetime import datetime

Base = declarative_base()

engine = create_engine(connection_string, echo=True)
"""
class User
    id int
    username str
    email str
    date_create datetime
"""

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    username = Column(String(25), nullable=False, unique=True)
    email = Column(String(80), nullable=False, unique=True)
    date_created = Column(DateTime, default=datetime.utcnow())
    
    def __repr__ (self):
        return f"<User username={self.username} email={self.email}>"
    
new_user = User(id=1, username="jonathan", email = "jone@hui.com")
print (new_user)

<User username=jonathan email=jone@hui.com>


In [73]:
#create database if not exists
Base.metadata.create_all(engine)

2022-11-28 20:48:20,384 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:20,385 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-11-28 20:48:20,385 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:20,385 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-11-28 20:48:20,386 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:20,386 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	username VARCHAR(25) NOT NULL, 
	email VARCHAR(80) NOT NULL, 
	date_created DATETIME, 
	PRIMARY KEY (id), 
	UNIQUE (username), 
	UNIQUE (email)
)


2022-11-28 20:48:20,386 INFO sqlalchemy.engine.Engine [no key 0.00019s] ()
2022-11-28 20:48:20,387 INFO sqlalchemy.engine.Engine COMMIT


session sirve para hacer las transacciones (lazy¿?)

In [74]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker()
local_session = Session(bind=engine)

# Insert

In [75]:
new_user = User(username="frank", email = "frank@sinatra.com")
local_session.add(new_user)
local_session.commit()

2022-11-28 20:48:20,489 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:20,490 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-28 20:48:20,490 INFO sqlalchemy.engine.Engine [generated in 0.00034s] ('frank', 'frank@sinatra.com', '2022-11-28 23:48:20.347057')
2022-11-28 20:48:20,491 INFO sqlalchemy.engine.Engine COMMIT


In [76]:
users = [
    {"username":"jhonny",
    "email":"jh@company.com"},
    {"username":"jerry",
    "email":"je@company.com"},
    {"username":"lucas",
    "email":"lu@company.com"},
    {"username":"igor",
    "email":"ig@company.com"},
    {"username":"gaby",
    "email":"gb@company.com"},
    {"username":"tom",
    "email":"tm@company.com"},
]

In [77]:
for u in users:
    new_user = User(username = u["username"], email = u["email"])
    local_session.add(new_user)    
    local_session.commit()

2022-11-28 20:48:20,576 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:20,576 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-28 20:48:20,577 INFO sqlalchemy.engine.Engine [cached since 0.08678s ago] ('jhonny', 'jh@company.com', '2022-11-28 23:48:20.347057')
2022-11-28 20:48:20,577 INFO sqlalchemy.engine.Engine COMMIT
2022-11-28 20:48:20,577 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:20,578 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-28 20:48:20,578 INFO sqlalchemy.engine.Engine [cached since 0.08807s ago] ('jerry', 'je@company.com', '2022-11-28 23:48:20.347057')
2022-11-28 20:48:20,578 INFO sqlalchemy.engine.Engine COMMIT
2022-11-28 20:48:20,579 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:20,579 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-28 20:48:20,5

# Query

In [78]:
users = local_session.query(User).all()
for user in users:
    print (user.username)

2022-11-28 20:48:20,614 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:20,615 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users
2022-11-28 20:48:20,615 INFO sqlalchemy.engine.Engine [generated in 0.00030s] ()
frank
jhonny
jerry
lucas
igor
gaby
tom


In [79]:
users = local_session.query(User).all()[:3]
for user in users:
    print (user.username)

2022-11-28 20:48:20,648 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users
2022-11-28 20:48:20,649 INFO sqlalchemy.engine.Engine [cached since 0.03392s ago] ()
frank
jhonny
jerry


# Update

In [80]:
#gaby
user = local_session.query(User).filter(User.username == "gaby").first()


2022-11-28 20:48:20,682 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users 
WHERE users.username = ?
 LIMIT ? OFFSET ?
2022-11-28 20:48:20,683 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ('gaby', 1, 0)


In [81]:
print (user)

<User username=gaby email=gb@company.com>


In [82]:
#updating

In [83]:
user

<User username=gaby email=gb@company.com>

In [84]:
user.username="gabriela"
user.email = "gabriela@company.com"

In [85]:
local_session.commit()

2022-11-28 20:48:20,832 INFO sqlalchemy.engine.Engine UPDATE users SET username=?, email=? WHERE users.id = ?
2022-11-28 20:48:20,832 INFO sqlalchemy.engine.Engine [generated in 0.00085s] ('gabriela', 'gabriela@company.com', 6)
2022-11-28 20:48:20,833 INFO sqlalchemy.engine.Engine COMMIT


# Delete

In [86]:
user_to_delete = local_session.query(User).filter(User.username == "gabriela").first()

2022-11-28 20:48:20,859 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:20,859 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users 
WHERE users.username = ?
 LIMIT ? OFFSET ?
2022-11-28 20:48:20,860 INFO sqlalchemy.engine.Engine [cached since 0.1774s ago] ('gabriela', 1, 0)


In [87]:
local_session.delete(user_to_delete)

In [88]:
local_session.commit()

2022-11-28 20:48:20,913 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2022-11-28 20:48:20,914 INFO sqlalchemy.engine.Engine [generated in 0.00051s] (6,)
2022-11-28 20:48:20,914 INFO sqlalchemy.engine.Engine COMMIT


# Order

In [89]:
users = local_session.query(User).order_by(User.username).all()

2022-11-28 20:48:20,947 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:20,948 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users ORDER BY users.username
2022-11-28 20:48:20,948 INFO sqlalchemy.engine.Engine [generated in 0.00037s] ()


In [90]:
users

[<User username=frank email=frank@sinatra.com>,
 <User username=igor email=ig@company.com>,
 <User username=jerry email=je@company.com>,
 <User username=jhonny email=jh@company.com>,
 <User username=lucas email=lu@company.com>,
 <User username=tom email=tm@company.com>]

In [91]:
from sqlalchemy import desc
users = local_session.query(User).order_by(desc(User.username)).all()
users

2022-11-28 20:48:21,006 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users ORDER BY users.username DESC
2022-11-28 20:48:21,007 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ()


[<User username=tom email=tm@company.com>,
 <User username=lucas email=lu@company.com>,
 <User username=jhonny email=jh@company.com>,
 <User username=jerry email=je@company.com>,
 <User username=igor email=ig@company.com>,
 <User username=frank email=frank@sinatra.com>]

In [92]:
import sqlalchemy
sqlalchemy.__version__

'1.4.43'

In [93]:
%reset -f

Otro Ejemplo

In [94]:
from sqlalchemy import create_engine, ForeignKey, Column, String, Integer, CHAR
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Persona(Base):
    __tablename__ = "personas"
    
    ssn = Column(Integer, primary_key=True)
    firstname = Column(String)
    lastname = Column(String)
    gender = Column(CHAR)
    age = Column(Integer)
    
    def __init__(self, ssn, firstname, lastname, gender, age):
        self.ssn = ssn
        self.firstname = firstname
        self.lastname = lastname
        self.gender = gender.upper()
        self.age = age
        
    def __repr__(self):
        return f"{self.ssn} {self.firstname} {self.lastname} ({self.gender}, {self.age})"

In [95]:
engine = create_engine("sqlite:///:memory:", echo=True)

In [96]:
Base.metadata.create_all(bind=engine)

2022-11-28 20:48:21,232 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:21,233 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("personas")
2022-11-28 20:48:21,234 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:21,234 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("personas")
2022-11-28 20:48:21,234 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:21,235 INFO sqlalchemy.engine.Engine 
CREATE TABLE personas (
	ssn INTEGER NOT NULL, 
	firstname VARCHAR, 
	lastname VARCHAR, 
	gender CHAR, 
	age INTEGER, 
	PRIMARY KEY (ssn)
)


2022-11-28 20:48:21,235 INFO sqlalchemy.engine.Engine [no key 0.00029s] ()
2022-11-28 20:48:21,236 INFO sqlalchemy.engine.Engine COMMIT


In [97]:
Session = sessionmaker(bind=engine)
session = Session()

In [98]:
tom = Persona(21878, "Mike", "Smith", "m", 35)
session.add(tom)
session.commit()

2022-11-28 20:48:21,310 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:21,311 INFO sqlalchemy.engine.Engine INSERT INTO personas (ssn, firstname, lastname, gender, age) VALUES (?, ?, ?, ?, ?)
2022-11-28 20:48:21,311 INFO sqlalchemy.engine.Engine [generated in 0.00035s] (21878, 'Mike', 'Smith', 'M', 35)
2022-11-28 20:48:21,312 INFO sqlalchemy.engine.Engine COMMIT


In [99]:
p1 = Persona(8219821, "Jhon", "Smith", "m", 31)
p2 = Persona(209130938, "Lucas", "Row", "m", 23)
p3 = Persona(99999, "Claudia", "López", "f", 45)
session.add_all([p1,p2,p3])

In [100]:
session.commit()

2022-11-28 20:48:21,431 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:21,432 INFO sqlalchemy.engine.Engine INSERT INTO personas (ssn, firstname, lastname, gender, age) VALUES (?, ?, ?, ?, ?)
2022-11-28 20:48:21,433 INFO sqlalchemy.engine.Engine [generated in 0.00034s] ((8219821, 'Jhon', 'Smith', 'M', 31), (209130938, 'Lucas', 'Row', 'M', 23), (99999, 'Claudia', 'López', 'F', 45))
2022-11-28 20:48:21,433 INFO sqlalchemy.engine.Engine COMMIT


In [101]:
    session.query(Persona).all()

2022-11-28 20:48:21,460 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:21,461 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas
2022-11-28 20:48:21,461 INFO sqlalchemy.engine.Engine [generated in 0.00026s] ()


[21878 Mike Smith (M, 35),
 99999 Claudia López (F, 45),
 8219821 Jhon Smith (M, 31),
 209130938 Lucas Row (M, 23)]

In [102]:
resultados = session.query(Persona)
print ("==")
for r in resultados:
    print (r)

==
2022-11-28 20:48:21,491 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas
2022-11-28 20:48:21,491 INFO sqlalchemy.engine.Engine [cached since 0.03027s ago] ()
21878 Mike Smith (M, 35)
99999 Claudia López (F, 45)
8219821 Jhon Smith (M, 31)
209130938 Lucas Row (M, 23)


In [103]:
nuevos = resultados.filter(Persona.age < 40).all()
print (nuevos)

2022-11-28 20:48:21,523 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas 
WHERE personas.age < ?
2022-11-28 20:48:21,523 INFO sqlalchemy.engine.Engine [generated in 0.00056s] (40,)
[21878 Mike Smith (M, 35), 8219821 Jhon Smith (M, 31), 209130938 Lucas Row (M, 23)]


# Relaciones

In [104]:
%reset -f

In [105]:
from sqlalchemy import create_engine, ForeignKey, Column, String, Integer, CHAR
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Persona(Base):
    __tablename__ = "personas"
    
    ssn = Column(Integer, primary_key=True)
    firstname = Column(String)
    lastname = Column(String)
    gender = Column(CHAR)
    age = Column(Integer)
    
    def __init__(self, ssn, firstname, lastname, gender, age):
        self.ssn = ssn
        self.firstname = firstname
        self.lastname = lastname
        self.gender = gender.upper()
        self.age = age
        
    def __repr__(self):
        return f"{self.ssn} {self.firstname} {self.lastname} ({self.gender}, {self.age})"
    
class Cosa(Base):
    __tablename__ = "cosas"
    
    tid = Column(Integer, primary_key=True)
    description = Column(String)
    owner = Column(Integer, ForeignKey("personas.ssn"))
    
    def __repr__(self):
        return f"{self.tid} {self.description} owned by {self.owner}"
    
engine = create_engine("sqlite:///:memory:", echo=True)
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()


2022-11-28 20:48:21,653 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:21,654 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("personas")
2022-11-28 20:48:21,654 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:21,654 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("personas")
2022-11-28 20:48:21,655 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:21,655 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cosas")
2022-11-28 20:48:21,655 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:21,655 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cosas")
2022-11-28 20:48:21,656 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:21,656 INFO sqlalchemy.engine.Engine 
CREATE TABLE personas (
	ssn INTEGER NOT NULL, 
	firstname VARCHAR, 
	lastname VARCHAR, 
	gender CHAR, 
	age INTEGER, 
	PRIMARY KEY (ssn)
)


2022-11-28 20:48:21,656 INFO sqlalchemy.engine.Engine [no key 0.00017s] ()
2022-11-28 20:48:21,657 INFO sqlalchemy

In [106]:
p0 = Persona(21878, "Mike", "Smith", "m", 35)
p1 = Persona(8219821, "Jhon", "Smith", "m", 31)
p2 = Persona(209130938, "Lucas", "Row", "m", 23)
p3 = Persona(99999, "Claudia", "López", "f", 45)
session.add_all([p0, p1,p2,p3])
session.commit()

2022-11-28 20:48:21,681 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:21,682 INFO sqlalchemy.engine.Engine INSERT INTO personas (ssn, firstname, lastname, gender, age) VALUES (?, ?, ?, ?, ?)
2022-11-28 20:48:21,682 INFO sqlalchemy.engine.Engine [generated in 0.00032s] ((21878, 'Mike', 'Smith', 'M', 35), (8219821, 'Jhon', 'Smith', 'M', 31), (209130938, 'Lucas', 'Row', 'M', 23), (99999, 'Claudia', 'López', 'F', 45))
2022-11-28 20:48:21,683 INFO sqlalchemy.engine.Engine COMMIT


In [107]:
t1 = Cosa(tid= 23893232, description = "Una cosa", owner = p0.ssn)

2022-11-28 20:48:21,707 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:21,708 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas 
WHERE personas.ssn = ?
2022-11-28 20:48:21,709 INFO sqlalchemy.engine.Engine [generated in 0.00034s] (21878,)


In [108]:
session.add(t1)
session.commit()

2022-11-28 20:48:21,740 INFO sqlalchemy.engine.Engine INSERT INTO cosas (tid, description, owner) VALUES (?, ?, ?)
2022-11-28 20:48:21,741 INFO sqlalchemy.engine.Engine [generated in 0.00041s] (23893232, 'Una cosa', 21878)
2022-11-28 20:48:21,741 INFO sqlalchemy.engine.Engine COMMIT


In [109]:
t2 = Cosa(description = "PS5", owner = p1.ssn)
t3 = Cosa(description = "Auto", owner = p0.ssn)
session.add_all([t2,t3])
session.commit()

2022-11-28 20:48:21,771 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:21,771 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas 
WHERE personas.ssn = ?
2022-11-28 20:48:21,771 INFO sqlalchemy.engine.Engine [cached since 0.06311s ago] (8219821,)
2022-11-28 20:48:21,772 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas 
WHERE personas.ssn = ?
2022-11-28 20:48:21,772 INFO sqlalchemy.engine.Engine [cached since 0.064s ago] (21878,)
2022-11-28 20:48:21,773 INFO sqlalchemy.engine.Engine INSERT INTO cosas (description, owner) VALUES (?, ?)
2022-11-28 20:48:21,773 INFO sqlalchemy.engine.Engine [generated in 0.00034s] ('PS5', 

In [110]:
t3

2022-11-28 20:48:21,799 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:21,800 INFO sqlalchemy.engine.Engine SELECT cosas.tid AS cosas_tid, cosas.description AS cosas_description, cosas.owner AS cosas_owner 
FROM cosas 
WHERE cosas.tid = ?
2022-11-28 20:48:21,800 INFO sqlalchemy.engine.Engine [generated in 0.00068s] (23893234,)


23893234 Auto owned by 21878

In [111]:
results = session.query(Cosa, Persona).filter(Cosa.owner == Persona.ssn).filter(Persona.firstname == "Mike")

In [112]:
for r in results:
    print (r)

2022-11-28 20:48:21,863 INFO sqlalchemy.engine.Engine SELECT cosas.tid AS cosas_tid, cosas.description AS cosas_description, cosas.owner AS cosas_owner, personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM cosas, personas 
WHERE cosas.owner = personas.ssn AND personas.firstname = ?
2022-11-28 20:48:21,864 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ('Mike',)
(23893232 Una cosa owned by 21878, 21878 Mike Smith (M, 35))
(23893234 Auto owned by 21878, 21878 Mike Smith (M, 35))


In [113]:
%reset -f

# Otro Ejemplo 
### 1. Partimos con DBAPI

In [114]:
import sqlite3

db_file = "local.db"
with sqlite3.connect(db_file) as conn:
    conn.execute(
        """CREATE TABLE IF NOT EXISTS test (
            col1 integer,
            col2 string
            )
            """
    )

In [115]:
with sqlite3.connect(db_file) as conn:
    conn.execute(
    """INSERT INTO test VALUES (:val_1, :val_2)""",
    {"val_1": 1, "val_2":2}
    )
    result = conn.execute("SELECT * FROM test").fetchall()
result

[(1, 2), (1, 2), (1, 2)]

El mecanismo de arriba es para evitar una inyección de SQL

In [116]:
type(result[0])

tuple

Igual es un sistema complicado, porque las queries son texto y si cambio el motor de BBDD tengo que cambiar el código y no ocupa características de Python

### 2. Usando SqlAlchemy

In [117]:
import sqlalchemy as sa

Metadata es el registro de las tablas y permite a SQLAlchemy entender como se conectan. Debe ser un objeto global y todas las tablas deben usar el mismo objeto metadata

In [118]:
meta = sa.MetaData()

Interpretamos la tabla en Python, se puede usar:

In [119]:
test_table = sa.Table("test", 
                   meta,
                   sa.Column("col1", sa.Integer),
                   sa.Column("col2", sa.String))

Todavía la tabla no tiene idea de la base de datos, para eso se usa el **engine**

Al crear el **engine** no se conecta a la DB, simplemente hace validación de la dirección (path, url) y prepara los dialectos para el motor, también se pueden poder muchas opciones de conexión.

In [120]:
#future da uso de las opciones de SQLAlchemy 2.0 que todavía está en beta
#echo escupe el SQL

engine = sa.create_engine("sqlite:///local.db", future=True)


Ahora podemos hacer consultas usando SA

In [121]:
sql = sa.select(test_table)

In [122]:
sql

In [123]:
print(sql)

SELECT test.col1, test.col2 
FROM test


In [124]:
test_table.c.col1 == "test"

In [125]:
print(test_table.c.col1 == "test")


test.col1 = :col1_1


In [126]:
print(sql.where(test_table.c.col1 == "test"))


SELECT test.col1, test.col2 
FROM test 
WHERE test.col1 = :col1_1


Para correr el SQL debemos establecer una conexión, es la primera vez que hacemos algo fuera de python /en esta vuelta)

In [127]:
with engine.connect() as conn:
    result = conn.execute(sql).all()
    
result

[(1, 2), (1, 2), (1, 2)]

In [128]:
type(result[0])

sqlalchemy.engine.row.Row

In [129]:
result[0].col1

1

In [130]:
dict(result[0])

{'col1': 1, 'col2': 2}

Se parece a lo de antes, pero mejorado

Usamos `.all()` para dar todos, pero se puede usar:
*   `all` todos los resultados
*   `one` solo uno y da excepción si no hay exactamente uno
*   `one_or_none` da uno y da excepción si hay más de uno
*   `first` el primero
*   `partitions(size)` en chunks de tamaño size
*   `yield_per(num)` solo si el motor soporta stream de resultados

### 3. SQLAlchemy Core
Vamos a crear las tablas, que recordemos no están unidas a ninguna base

Vamos a usar unos datos daneses de estacionamientos, porque podemos hacerlo
https://www.opendata.dk/city-of-copenhagen/parkeringstaelling-i-zoner

In [131]:
import sqlalchemy as sa

meta = sa.MetaData()

fkt_parking = sa.Table("fkt_parking", 
                       meta, 
                       sa.Column("id", sa.Integer, primary_key=True),
                       sa.Column("area_id", sa.Integer, sa.ForeignKey("dim_area.area_id")), 
                       sa.Column("year_month", sa.VARCHAR(20)),
                       sa.Column("count_type", sa.Integer, sa.ForeignKey("dim_parking_types.type_id")),
                       sa.Column("count", sa.Integer),
                       sa.Column("hour", sa.Integer)
                      )

dim_area = sa.Table("dim_area", 
                    meta,
                    sa.Column("area_id", sa.Integer, primary_key=True),
                    sa.Column("city", sa.VARCHAR(50)),
                    sa.Column("street_name", sa.VARCHAR(200)),
                    sa.Column("postnr", sa.VARCHAR(4)),
                    sa.Column("nr", sa.Integer)
                   )
                       
dim_parking_types = sa.Table("dim_parking_types",
                             meta,
                             sa.Column("type_id", sa.Integer, primary_key=True),
                             sa.Column("name", sa.VARCHAR(50), unique=True)
                            )

In [132]:
conn_string = "sqlite:///parking.db"

In [133]:
meta.tables


FacadeDict({'fkt_parking': Table('fkt_parking', MetaData(), Column('id', Integer(), table=<fkt_parking>, primary_key=True, nullable=False), Column('area_id', Integer(), ForeignKey('dim_area.area_id'), table=<fkt_parking>), Column('year_month', VARCHAR(length=20), table=<fkt_parking>), Column('count_type', Integer(), ForeignKey('dim_parking_types.type_id'), table=<fkt_parking>), Column('count', Integer(), table=<fkt_parking>), Column('hour', Integer(), table=<fkt_parking>), schema=None), 'dim_area': Table('dim_area', MetaData(), Column('area_id', Integer(), table=<dim_area>, primary_key=True, nullable=False), Column('city', VARCHAR(length=50), table=<dim_area>), Column('street_name', VARCHAR(length=200), table=<dim_area>), Column('postnr', VARCHAR(length=4), table=<dim_area>), Column('nr', Integer(), table=<dim_area>), schema=None), 'dim_parking_types': Table('dim_parking_types', MetaData(), Column('type_id', Integer(), table=<dim_parking_types>, primary_key=True, nullable=False), Colum

In [134]:
engine = sa.create_engine(conn_string, future=True, echo=True)


Como metadata sabe todo de nuestras tablas, le podemos pedir que las cree, conoce las dependencias, las relaciones y lo hace todo mágicamente

In [135]:
meta.create_all(engine)

2022-11-28 20:48:22,925 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:22,925 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fkt_parking")
2022-11-28 20:48:22,926 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:22,926 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("fkt_parking")
2022-11-28 20:48:22,926 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:22,927 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("dim_area")
2022-11-28 20:48:22,927 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:22,927 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("dim_area")
2022-11-28 20:48:22,927 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:22,928 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("dim_parking_types")
2022-11-28 20:48:22,928 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 20:48:22,928 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("dim_parking_types")
2022-11-28 20:48:22,928 INFO sqlalch

#### 3.1 Insert

In [136]:
insert_sql = sa.insert(dim_parking_types).values(type_id=0, name="legal")


In [137]:
print(insert_sql)

INSERT INTO dim_parking_types (type_id, name) VALUES (:type_id, :name)


Acá no hay espacio para la inyección y lo hace solo.

Esto es sólo una instrucción, hay que conectarse a la base de datos para que sirva de algo útil

In [138]:
with engine.connect() as conn:
    conn.execute(insert_sql)

2022-11-28 20:48:23,051 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:23,052 INFO sqlalchemy.engine.Engine INSERT INTO dim_parking_types (type_id, name) VALUES (?, ?)
2022-11-28 20:48:23,052 INFO sqlalchemy.engine.Engine [generated in 0.00076s] (0, 'legal')
2022-11-28 20:48:23,053 INFO sqlalchemy.engine.Engine ROLLBACK


Si miramos la base de datos, nada ha pasado.... esto es porque si no hacemos commit los cambios se van a deshacer al salir del context manager

Se ve en la última linea que hace eso

In [139]:
with engine.connect() as conn:
    conn.execute(insert_sql)
    conn.commit()

2022-11-28 20:48:23,110 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:23,110 INFO sqlalchemy.engine.Engine INSERT INTO dim_parking_types (type_id, name) VALUES (?, ?)
2022-11-28 20:48:23,111 INFO sqlalchemy.engine.Engine [cached since 0.05951s ago] (0, 'legal')
2022-11-28 20:48:23,112 INFO sqlalchemy.engine.Engine COMMIT


Ahora borremos ese dato

In [140]:
delete_sql = dim_parking_types.delete().where(dim_parking_types.c.type_id == 0)
print (delete_sql)

DELETE FROM dim_parking_types WHERE dim_parking_types.type_id = :type_id_1


In [141]:
with engine.connect() as conn:
    conn.execute(delete_sql)
    conn.commit()

2022-11-28 20:48:23,189 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:23,189 INFO sqlalchemy.engine.Engine DELETE FROM dim_parking_types WHERE dim_parking_types.type_id = ?
2022-11-28 20:48:23,189 INFO sqlalchemy.engine.Engine [generated in 0.00064s] (0,)
2022-11-28 20:48:23,190 INFO sqlalchemy.engine.Engine COMMIT


In [142]:
with engine.connect() as conn:
    conn.execute(insert_sql)
    conn.commit()
    conn.execute(delete_sql)
    conn.rollback()
    
#rollback hecha marcha atrás, lo que está con commit queda

2022-11-28 20:48:23,223 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:23,224 INFO sqlalchemy.engine.Engine INSERT INTO dim_parking_types (type_id, name) VALUES (?, ?)
2022-11-28 20:48:23,224 INFO sqlalchemy.engine.Engine [cached since 0.1727s ago] (0, 'legal')
2022-11-28 20:48:23,225 INFO sqlalchemy.engine.Engine COMMIT
2022-11-28 20:48:23,225 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:23,226 INFO sqlalchemy.engine.Engine DELETE FROM dim_parking_types WHERE dim_parking_types.type_id = ?
2022-11-28 20:48:23,226 INFO sqlalchemy.engine.Engine [cached since 0.03712s ago] (0,)
2022-11-28 20:48:23,226 INFO sqlalchemy.engine.Engine ROLLBACK


#### 3.2 Unit of Work

SA usa unit of work , esto quiere decir que espera que preparemos todos los cambios y de ahí lo enviemos a la BD de una, al mismo tiempo. Esto ayuda a SA a optimizar el resultado y la comunicación con la BD. 

El usar commit es parte de esto y ayuda a tener más control. 

Esto también significa que SA está generalmente trabajando con transacciones, las que podemos manejar explícitamente

In [143]:
with engine.connect() as conn:
    conn.execute(delete_sql)
    conn.commit()
    transaction = conn.begin()
    conn.execute(insert_sql)
    transaction.rollback()

2022-11-28 20:48:23,259 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:23,259 INFO sqlalchemy.engine.Engine DELETE FROM dim_parking_types WHERE dim_parking_types.type_id = ?
2022-11-28 20:48:23,260 INFO sqlalchemy.engine.Engine [cached since 0.07098s ago] (0,)
2022-11-28 20:48:23,260 INFO sqlalchemy.engine.Engine COMMIT
2022-11-28 20:48:23,261 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:23,261 INFO sqlalchemy.engine.Engine INSERT INTO dim_parking_types (type_id, name) VALUES (?, ?)
2022-11-28 20:48:23,261 INFO sqlalchemy.engine.Engine [cached since 0.2099s ago] (0, 'legal')
2022-11-28 20:48:23,262 INFO sqlalchemy.engine.Engine ROLLBACK


Se puede hacer así también

In [144]:
with engine.begin() as conn:
    conn.execute(insert_sql)
    conn.execute(delete_sql)

2022-11-28 20:48:23,292 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:23,292 INFO sqlalchemy.engine.Engine INSERT INTO dim_parking_types (type_id, name) VALUES (?, ?)
2022-11-28 20:48:23,293 INFO sqlalchemy.engine.Engine [cached since 0.2415s ago] (0, 'legal')
2022-11-28 20:48:23,294 INFO sqlalchemy.engine.Engine DELETE FROM dim_parking_types WHERE dim_parking_types.type_id = ?
2022-11-28 20:48:23,294 INFO sqlalchemy.engine.Engine [cached since 0.1054s ago] (0,)
2022-11-28 20:48:23,295 INFO sqlalchemy.engine.Engine COMMIT


Acá hace el commit solo al salir del context manager. Si hay un error entonces la transacción hace rollback

In [145]:
with engine.begin() as conn:
    conn.execute(insert_sql)
    raise Exception("Algo malo pasó")

2022-11-28 20:48:23,342 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:23,343 INFO sqlalchemy.engine.Engine INSERT INTO dim_parking_types (type_id, name) VALUES (?, ?)
2022-11-28 20:48:23,344 INFO sqlalchemy.engine.Engine [cached since 0.293s ago] (0, 'legal')
2022-11-28 20:48:23,346 INFO sqlalchemy.engine.Engine ROLLBACK


Exception: Algo malo pasó

#### 3.3 Insertar muchas filas

In [146]:
import csv
import pathlib

In [147]:
data = [(pathlib.Path("data/dim_parking_types.csv"), dim_parking_types),
        (pathlib.Path("data/dim_area.csv"), dim_area),
        (pathlib.Path("data/fkt_parking.csv"), fkt_parking)]

In [148]:
data

[(PosixPath('data/dim_parking_types.csv'),
  Table('dim_parking_types', MetaData(), Column('type_id', Integer(), table=<dim_parking_types>, primary_key=True, nullable=False), Column('name', VARCHAR(length=50), table=<dim_parking_types>), schema=None)),
 (PosixPath('data/dim_area.csv'),
  Table('dim_area', MetaData(), Column('area_id', Integer(), table=<dim_area>, primary_key=True, nullable=False), Column('city', VARCHAR(length=50), table=<dim_area>), Column('street_name', VARCHAR(length=200), table=<dim_area>), Column('postnr', VARCHAR(length=4), table=<dim_area>), Column('nr', Integer(), table=<dim_area>), schema=None)),
 (PosixPath('data/fkt_parking.csv'),
  Table('fkt_parking', MetaData(), Column('id', Integer(), table=<fkt_parking>, primary_key=True, nullable=False), Column('area_id', Integer(), ForeignKey('dim_area.area_id'), table=<fkt_parking>), Column('year_month', VARCHAR(length=20), table=<fkt_parking>), Column('count_type', Integer(), ForeignKey('dim_parking_types.type_id'),

In [149]:
#Corramos todo en una Unit of Work en una transacción
with engine.begin() as conn:
    for data_file, table in data:
        #leyendo CSV
        with data_file.open(encoding="utf-8") as f:
            rows = list(csv.DictReader(f))
        sql = table.insert()
        conn.execute(sql, parameters=rows)

2022-11-28 20:48:37,608 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:37,609 INFO sqlalchemy.engine.Engine INSERT INTO dim_parking_types (type_id, name) VALUES (?, ?)
2022-11-28 20:48:37,609 INFO sqlalchemy.engine.Engine [generated in 0.00023s] (('0', 'legal'), ('1', 'counted'))
2022-11-28 20:48:37,611 INFO sqlalchemy.engine.Engine INSERT INTO dim_area (area_id, city, street_name, postnr, nr) VALUES (?, ?, ?, ?, ?)
2022-11-28 20:48:37,611 INFO sqlalchemy.engine.Engine [generated in 0.00037s] (('1', 'Løgumkloster', 'Sommersted', '3886', '7'), ('5', 'Søborg', 'Kålager', '4873', '78'), ('2', 'Glejbjerg', 'Hannemanns', '6024', '2'), ('12', 'Gilleleje', 'Australiens', '5549', '5'), ('3', 'Haderup', 'Drechsels', '2909', '112'), ('4', 'Værløse', 'Hauser', '2196', '2'), ('6', 'Bøvlingbjerg', 'Tikøb', '9594', '17'), ('9', 'Vildbjerg', 'Hyldebær', '4767', '233')  ... displaying 10 of 103 total bound parameter sets ...  ('110', 'Horslunde', 'Suensons', '6244', '80'), ('111', 'Ve

Cuando pasamos una lista de diccionarios a paramters, SA sabe usar el método `.executemany()`de la libreria DB-API library, que optimiza para ingresar múltiples filas

#### 3.4 Usando lógica

Busquemos solo en la ciudad de Gilleleje


In [150]:
sql = sa.select([fkt_parking.c.count, fkt_parking.c.hour, dim_area]).join(dim_area).where(dim_area.c.city == "Gilleleje")
print (sql)

SELECT fkt_parking.count, fkt_parking.hour, dim_area.area_id, dim_area.city, dim_area.street_name, dim_area.postnr, dim_area.nr 
FROM fkt_parking JOIN dim_area ON dim_area.area_id = fkt_parking.area_id 
WHERE dim_area.city = :city_1


In [151]:
with engine.connect() as conn:
    results = conn.execute(sql).all()

2022-11-28 20:48:37,697 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:37,697 INFO sqlalchemy.engine.Engine SELECT fkt_parking.count, fkt_parking.hour, dim_area.area_id, dim_area.city, dim_area.street_name, dim_area.postnr, dim_area.nr 
FROM fkt_parking JOIN dim_area ON dim_area.area_id = fkt_parking.area_id 
WHERE dim_area.city = ?
2022-11-28 20:48:37,697 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ('Gilleleje',)
2022-11-28 20:48:37,699 INFO sqlalchemy.engine.Engine ROLLBACK


In [152]:
[dict(row) for row in results[:2]]

[{'count': 1004,
  'hour': 12,
  'area_id': 12,
  'city': 'Gilleleje',
  'street_name': 'Australiens',
  'postnr': '5549',
  'nr': 5},
 {'count': 1018,
  'hour': 12,
  'area_id': 12,
  'city': 'Gilleleje',
  'street_name': 'Australiens',
  'postnr': '5549',
  'nr': 5}]

In [153]:
import pandas as pd
pd.DataFrame(results)

,count,hour,area_id,city,street_name,postnr,nr
0,1004,12,12,Gilleleje,Australiens,5549,5
1,1018,12,12,Gilleleje,Australiens,5549,5
2,1019,12,12,Gilleleje,Australiens,5549,5
3,1028,12,12,Gilleleje,Australiens,5549,5
4,1032,12,12,Gilleleje,Australiens,5549,5
...,...,...,...,...,...,...,...
109,769,22,12,Gilleleje,Australiens,5549,5
110,766,22,12,Gilleleje,Australiens,5549,5
111,737,22,12,Gilleleje,Australiens,5549,5
112,743,22,12,Gilleleje,Australiens,5549,5


In [154]:
#alternativamente 
with engine.connect() as conn:
    df = pd.read_sql(sql, conn)
df

2022-11-28 20:48:38,199 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:38,200 INFO sqlalchemy.engine.Engine SELECT fkt_parking.count, fkt_parking.hour, dim_area.area_id, dim_area.city, dim_area.street_name, dim_area.postnr, dim_area.nr 
FROM fkt_parking JOIN dim_area ON dim_area.area_id = fkt_parking.area_id 
WHERE dim_area.city = ?
2022-11-28 20:48:38,201 INFO sqlalchemy.engine.Engine [cached since 0.504s ago] ('Gilleleje',)
2022-11-28 20:48:38,205 INFO sqlalchemy.engine.Engine ROLLBACK


,count,hour,area_id,city,street_name,postnr,nr
0,1004,12,12,Gilleleje,Australiens,5549,5
1,1018,12,12,Gilleleje,Australiens,5549,5
2,1019,12,12,Gilleleje,Australiens,5549,5
3,1028,12,12,Gilleleje,Australiens,5549,5
4,1032,12,12,Gilleleje,Australiens,5549,5
...,...,...,...,...,...,...,...
109,769,22,12,Gilleleje,Australiens,5549,5
110,766,22,12,Gilleleje,Australiens,5549,5
111,737,22,12,Gilleleje,Australiens,5549,5
112,743,22,12,Gilleleje,Australiens,5549,5


Ahora calculemos la suma de cuentas por hora por ciudad

`func` es para funciones de sql, en este caso la suma

In [155]:
sql = (
    sa.select([sa.func.sum(fkt_parking.c.count).label("total_count"), 
               dim_area.c.city,
               fkt_parking.c.hour])
    .join(dim_area)
    .group_by(dim_area.c.city, fkt_parking.c.hour)
    )


print(sql)

SELECT sum(fkt_parking.count) AS total_count, dim_area.city, fkt_parking.hour 
FROM fkt_parking JOIN dim_area ON dim_area.area_id = fkt_parking.area_id GROUP BY dim_area.city, fkt_parking.hour


In [156]:
with engine.connect() as conn:
    result = conn.execute(sql)
    df = pd.DataFrame(result)
df

2022-11-28 20:48:38,278 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:38,278 INFO sqlalchemy.engine.Engine SELECT sum(fkt_parking.count) AS total_count, dim_area.city, fkt_parking.hour 
FROM fkt_parking JOIN dim_area ON dim_area.area_id = fkt_parking.area_id GROUP BY dim_area.city, fkt_parking.hour
2022-11-28 20:48:38,279 INFO sqlalchemy.engine.Engine [generated in 0.00049s] ()
2022-11-28 20:48:38,283 INFO sqlalchemy.engine.Engine ROLLBACK


,total_count,city,hour
0,14194,Allinge,12
1,13606,Allinge,17
2,13484,Allinge,22
3,33608,Ansager,12
4,35831,Ansager,17
...,...,...,...
271,27514,Vildbjerg,17
272,29047,Vildbjerg,22
273,29857,Værløse,12
274,29883,Værløse,17


#### 3.5 Refactorizando

Está quedando todo muy largo, y con repeticiones. Como es python podemos crear funciones y variables para manipular el SQL

In [157]:
table = fkt_parking.join(dim_area)
print (table)

fkt_parking JOIN dim_area ON dim_area.area_id = fkt_parking.area_id


In [158]:
total_count = sa.func.sum(fkt_parking.c.count).label("total_count")
print (total_count)

sum(fkt_parking.count)


In [159]:
dimension_cols = [dim_area.c.city, fkt_parking.c.hour]
print (dimension_cols)

[Column('city', VARCHAR(length=50), table=<dim_area>), Column('hour', Integer(), table=<fkt_parking>)]


In [160]:
sql = sa.select([total_count, *dimension_cols]).select_from(table).group_by(*dimension_cols)


Esto es el mismo código que antes, pero se ve más legible

    sql = (sa.select([sa.func.sum(fkt_parking.c.count).label("total_count"), 
                  dim_area.c.city,
                  fkt_parking.c.hour])
         .join(dim_area)
         .group_by(dim_area.c.city, fkt_parking.c.hour))

#### 3.6 cambios on-the-fly

In [161]:
#Agregar una columna adicional al groupby
print(sql.group_by(fkt_parking.c.count_type))

SELECT sum(fkt_parking.count) AS total_count, dim_area.city, fkt_parking.hour 
FROM fkt_parking JOIN dim_area ON dim_area.area_id = fkt_parking.area_id GROUP BY dim_area.city, fkt_parking.hour, fkt_parking.count_type


In [162]:
# Columna adicional al select
print(sql.add_columns(fkt_parking.c.count_type))

SELECT sum(fkt_parking.count) AS total_count, dim_area.city, fkt_parking.hour, fkt_parking.count_type 
FROM fkt_parking JOIN dim_area ON dim_area.area_id = fkt_parking.area_id GROUP BY dim_area.city, fkt_parking.hour


In [163]:
# Esto no alteró el SQL original
print(sql)

SELECT sum(fkt_parking.count) AS total_count, dim_area.city, fkt_parking.hour 
FROM fkt_parking JOIN dim_area ON dim_area.area_id = fkt_parking.area_id GROUP BY dim_area.city, fkt_parking.hour


In [164]:
with engine.connect() as conn:
    # Dynamically add a limit statement
    result = conn.execute(sql.limit(10))
    df = pd.DataFrame(result)
df


2022-11-28 20:48:38,516 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:38,516 INFO sqlalchemy.engine.Engine SELECT sum(fkt_parking.count) AS total_count, dim_area.city, fkt_parking.hour 
FROM fkt_parking JOIN dim_area ON dim_area.area_id = fkt_parking.area_id GROUP BY dim_area.city, fkt_parking.hour
 LIMIT ? OFFSET ?
2022-11-28 20:48:38,516 INFO sqlalchemy.engine.Engine [generated in 0.00076s] (10, 0)
2022-11-28 20:48:38,519 INFO sqlalchemy.engine.Engine ROLLBACK


,total_count,city,hour
0,14194,Allinge,12
1,13606,Allinge,17
2,13484,Allinge,22
3,33608,Ansager,12
4,35831,Ansager,17
5,37953,Ansager,22
6,40718,Asaa,12
7,37855,Asaa,17
8,39023,Asaa,22
9,36405,Beder,12


Ahora calculemos la tasa de ocupación. Se podría definir una tabla intermedia *(CTE (Common Table Expression)* o una subquery

In [165]:
# Reusamos el select básico
base_select = sa.select([fkt_parking.c.count, fkt_parking.c.area_id, fkt_parking.c.year_month, fkt_parking.c.hour]).join(dim_parking_types)

In [166]:
print (base_select)

SELECT fkt_parking.count, fkt_parking.area_id, fkt_parking.year_month, fkt_parking.hour 
FROM fkt_parking JOIN dim_parking_types ON dim_parking_types.type_id = fkt_parking.count_type


In [167]:
# Creamos un CTE que se llama available_spaces filtrado en  parking_types
available_spaces = base_select.where(dim_parking_types.c.name == "legal").cte("available_spaces")
print (available_spaces)

SELECT fkt_parking.count, fkt_parking.area_id, fkt_parking.year_month, fkt_parking.hour 
FROM fkt_parking JOIN dim_parking_types ON dim_parking_types.type_id = fkt_parking.count_type 
WHERE dim_parking_types.name = :name_1


In [168]:
# Otro CTE llamado  occupied spaces filtrado en parking_types
counted_spaces = base_select.where(dim_parking_types.c.name == "counted").cte("occupied_spaces")
print (counted_spaces)

SELECT fkt_parking.count, fkt_parking.area_id, fkt_parking.year_month, fkt_parking.hour 
FROM fkt_parking JOIN dim_parking_types ON dim_parking_types.type_id = fkt_parking.count_type 
WHERE dim_parking_types.name = :name_1


In [169]:
# Definimos la métrica
occupancy_rate = (100 * sa.cast(counted_spaces.c.count, sa.Float) / available_spaces.c.count).label("occupancy_rate")

In [170]:
print (occupancy_rate)

(:param_1 * CAST(occupied_spaces.count AS FLOAT)) / available_spaces.count


In [171]:
# Definimos el join, los CTE no tienen claves foraneas que SA pueda usar para inferir la relación
cte_join_condition = sa.and_(counted_spaces.c.area_id == available_spaces.c.area_id,
                        counted_spaces.c.year_month == available_spaces.c.year_month,
                        counted_spaces.c.hour == available_spaces.c.hour
                        )
print (cte_join_condition)

occupied_spaces.area_id = available_spaces.area_id AND occupied_spaces.year_month = available_spaces.year_month AND occupied_spaces.hour = available_spaces.hour


In [172]:
# Creamos el join
joined_ctes = available_spaces.join(counted_spaces, onclause=cte_join_condition)
print(joined_ctes)

SELECT fkt_parking.count AS count, fkt_parking.area_id AS area_id, fkt_parking.year_month AS year_month, fkt_parking.hour AS hour 
FROM fkt_parking JOIN dim_parking_types ON dim_parking_types.type_id = fkt_parking.count_type 
WHERE dim_parking_types.name = :name_1 JOIN SELECT fkt_parking.count AS count, fkt_parking.area_id AS area_id, fkt_parking.year_month AS year_month, fkt_parking.hour AS hour 
FROM fkt_parking JOIN dim_parking_types ON dim_parking_types.type_id = fkt_parking.count_type 
WHERE dim_parking_types.name = :name_2 ON occupied_spaces.area_id = available_spaces.area_id AND occupied_spaces.year_month = available_spaces.year_month AND occupied_spaces.hour = available_spaces.hour


In [173]:
sql = (sa.select([occupancy_rate, dim_area.c.city, available_spaces.c.year_month, available_spaces.c.hour])
       .select_from(joined_ctes)
       .join(dim_area) # SQLAlchemy puede inferir las claves foráneas a través del CTE
      )
print(sql)

WITH available_spaces AS 
(SELECT fkt_parking.count AS count, fkt_parking.area_id AS area_id, fkt_parking.year_month AS year_month, fkt_parking.hour AS hour 
FROM fkt_parking JOIN dim_parking_types ON dim_parking_types.type_id = fkt_parking.count_type 
WHERE dim_parking_types.name = :name_1), 
occupied_spaces AS 
(SELECT fkt_parking.count AS count, fkt_parking.area_id AS area_id, fkt_parking.year_month AS year_month, fkt_parking.hour AS hour 
FROM fkt_parking JOIN dim_parking_types ON dim_parking_types.type_id = fkt_parking.count_type 
WHERE dim_parking_types.name = :name_2)
 SELECT (:param_1 * CAST(occupied_spaces.count AS FLOAT)) / available_spaces.count AS occupancy_rate, dim_area.city, available_spaces.year_month, available_spaces.hour 
FROM available_spaces JOIN occupied_spaces ON occupied_spaces.area_id = available_spaces.area_id AND occupied_spaces.year_month = available_spaces.year_month AND occupied_spaces.hour = available_spaces.hour JOIN dim_area ON dim_area.area_id = occupi

In [174]:
with engine.connect() as conn:
    df = pd.read_sql(sql.order_by(occupancy_rate.desc()), conn)
df

2022-11-28 20:48:38,911 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:38,912 INFO sqlalchemy.engine.Engine WITH available_spaces AS 
(SELECT fkt_parking.count AS count, fkt_parking.area_id AS area_id, fkt_parking.year_month AS year_month, fkt_parking.hour AS hour 
FROM fkt_parking JOIN dim_parking_types ON dim_parking_types.type_id = fkt_parking.count_type 
WHERE dim_parking_types.name = ?), 
occupied_spaces AS 
(SELECT fkt_parking.count AS count, fkt_parking.area_id AS area_id, fkt_parking.year_month AS year_month, fkt_parking.hour AS hour 
FROM fkt_parking JOIN dim_parking_types ON dim_parking_types.type_id = fkt_parking.count_type 
WHERE dim_parking_types.name = ?)
 SELECT (? * CAST(occupied_spaces.count AS FLOAT)) / available_spaces.count AS occupancy_rate, dim_area.city, available_spaces.year_month, available_spaces.hour 
FROM available_spaces JOIN occupied_spaces ON occupied_spaces.area_id = available_spaces.area_id AND occupied_spaces.year_month = available_spa

,occupancy_rate,city,year_month,hour
0,355.882353,Skanderborg,2012-03,17
1,317.647059,Skanderborg,2012-09,17
2,294.117647,Skanderborg,2012-03,22
3,286.153846,Skanderborg,2015-09,22
4,283.823529,Skanderborg,2012-09,22
...,...,...,...,...
4453,2.064220,Sæby,2015-09,22
4454,1.927711,Sæby,2016-09,22
4455,1.465201,Sæby,2013-03,22
4456,1.282051,Sæby,2013-09,22


Ejercicio: Filtrar las que tienen ocupación sobre 100

In [175]:
with engine.connect() as conn:
    df = pd.read_sql(sql.where(occupancy_rate > 100).order_by(occupancy_rate.desc()), conn)
df

2022-11-28 20:48:38,963 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 20:48:38,964 INFO sqlalchemy.engine.Engine WITH available_spaces AS 
(SELECT fkt_parking.count AS count, fkt_parking.area_id AS area_id, fkt_parking.year_month AS year_month, fkt_parking.hour AS hour 
FROM fkt_parking JOIN dim_parking_types ON dim_parking_types.type_id = fkt_parking.count_type 
WHERE dim_parking_types.name = ?), 
occupied_spaces AS 
(SELECT fkt_parking.count AS count, fkt_parking.area_id AS area_id, fkt_parking.year_month AS year_month, fkt_parking.hour AS hour 
FROM fkt_parking JOIN dim_parking_types ON dim_parking_types.type_id = fkt_parking.count_type 
WHERE dim_parking_types.name = ?)
 SELECT (? * CAST(occupied_spaces.count AS FLOAT)) / available_spaces.count AS occupancy_rate, dim_area.city, available_spaces.year_month, available_spaces.hour 
FROM available_spaces JOIN occupied_spaces ON occupied_spaces.area_id = available_spaces.area_id AND occupied_spaces.year_month = available_spa

,occupancy_rate,city,year_month,hour
0,355.882353,Skanderborg,2012-03,17
1,317.647059,Skanderborg,2012-09,17
2,294.117647,Skanderborg,2012-03,22
3,286.153846,Skanderborg,2015-09,22
4,283.823529,Skanderborg,2012-09,22
...,...,...,...,...
785,100.094518,Holeby,2021-03,22
786,100.093545,Ryslinge,2012-09,22
787,100.089366,Nørre Aaby,2016-09,22
788,100.075643,Holstebro,2018-09,22


Core SQL es parecido a SQL pero con las ventajas de un lenguaje de programación

### 4. SQLAlchemy ORM

ORM es Object Relational Mapper, y es una capa que mapea la BD a objetos de pythons. Esto puede ser a expensas de cierta flexibilidad y transparencia respect al SQL subyacente.

Como regla general
*   Core sirve mejor para queries analiticas donde se espera obtener muchas filas
*   ORM es mejor para aplicaciones donde solo necesitamos unas cuanas filas cada vez

#### 4.1 Definiendo tablas

In [176]:
import sqlalchemy as sa
from sqlalchemy.orm import declarative_base

Base = declarative_base() 

class MyClass(Base):
    __tablename__ = "demo_table"
    
    #Hay que definir al menos una clave primaria
    class_id: int = sa.Column(sa.Integer, primary_key=True)
    name: str = sa.Column(sa.String)

Uno de los cambios en SA 2.0 es la habilidad de registrar clases con un decorador, lo que parece más en línea con clases basadas en `dataclass`y `attrs`


In [177]:
from sqlalchemy.orm import registry
import enum

mapper_registry = registry()

Dado el `registry` podemos definir clases que definen el modelo de dato. Los type hints son opcionales

Estas son clases regulares, entonces podemos agregar un  `__repr__` por ejemplo. 

In [178]:
@mapper_registry.mapped
class Address:
    __tablename__ = "addresses"
    
    address_id: int = sa.Column(sa.Integer, primary_key=True)
    street_name: str = sa.Column(sa.VARCHAR(50))
    street_number: int = sa.Column(sa.Integer)
    postnr: str = sa.Column(sa.VARCHAR(4))
    
    # Como es un simplemente una clase puedo poner un repr
    def __repr__(self):
        return f"{self.street_name} street_number={self.street_number} postnr={self.postnr}>"

La capa ORM genera una Tabla SA y la fija al atributo `__table__` , que es el mismo que vimos en Core



In [179]:
Address.__table__

Table('addresses', MetaData(), Column('address_id', Integer(), table=<addresses>, primary_key=True, nullable=False), Column('street_name', VARCHAR(length=50), table=<addresses>), Column('street_number', Integer(), table=<addresses>), Column('postnr', VARCHAR(length=4), table=<addresses>), schema=None)

No definimos un `__init__` porque SA genera uno automáticamente. Podríamos agregarlo si queremos y se puede agregar más lógica.

Vamos a agregar un objeto `Purchase` y otro `Customer`, con sus relaciones

In [180]:
import decimal
from sqlalchemy.orm import relationship

@mapper_registry.mapped
class Purchase:
    __tablename__ = "purchases"
    __table_args__ = {"extend_existing": True}
    
    purchase_id: int = sa.Column(sa.Integer, primary_key=True)
    item_name: str = sa.Column(sa.VARCHAR(200))
    price: decimal.Decimal = sa.Column(sa.Numeric(19, 4))
    user_id: int = sa.Column(sa.Integer, sa.ForeignKey("customers.customer_id"))
    
    def __repr__(self):
        return f"{self.item_name}>"

Podemos usar tipos nativos de python como enums y decimals, SA los va a convertir a los tipos que entienda la BD



In [181]:
class StatusEnum(str, enum.Enum):
    gold = "gold"
    silver = "silver"
    bronze = "bronze"
    
@mapper_registry.mapped
class Customer:
    __tablename__ = "customers"
    __table_args__ = {"extend_existing": True}
    
    customer_id: int = sa.Column(sa.Integer, primary_key=True)
    name: str = sa.Column(sa.VARCHAR(50), unique=True)
    status: str = sa.Column(sa.Enum(StatusEnum))
    address_id: int = sa.Column(sa.Integer, sa.ForeignKey("addresses.address_id"))
    
    # One-to-one relationship
    address: Address = relationship("Address", backref="customer")
    
    # One-to-many
    purchases: list[Purchase] = relationship("Purchase", backref="customer")
    
    def __repr__(self):
        return f"{self.name}>"

#### 4.2 Relaciones

In [ ]:
conn_string = "sqlite:///parking_orm.db"
engine = sa.create_engine(conn_string, future=True, echo=True)


ORM está construido sobre **Core**, podemos usar el engine y la metadata como antes

In [182]:
mapper_registry.metadata.create_all(engine)


2022-11-28 22:22:24,456 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:22:24,459 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2022-11-28 22:22:24,460 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 22:22:24,462 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("addresses")
2022-11-28 22:22:24,463 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 22:22:24,466 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("purchases")
2022-11-28 22:22:24,467 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 22:22:24,469 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("purchases")
2022-11-28 22:22:24,470 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 22:22:24,471 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customers")
2022-11-28 22:22:24,471 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 22:22:24,473 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("customers")
2022-11-28 22:22:24,476 INFO sqlalchemy.engine.Engine 

In [183]:
john = Customer(name="John", status=StatusEnum.gold)
jane = Customer(name="Jane", status=StatusEnum.bronze)

#### 4.3 Session
En ORM usamos Session en vez de conexión. La session sabe como trabajar con las clases de ORM, y sirve como mapa local de las instancias, manteniendo registro de que instancias han cambiado , cuales osn nuevas, etc.


In [184]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    session.add(john)
    session.add(jane)
    #hay que hace commit
    session.commit()

2022-11-28 22:24:58,465 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:24:58,466 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, status, address_id) VALUES (?, ?, ?)
2022-11-28 22:24:58,467 INFO sqlalchemy.engine.Engine [generated in 0.00049s] ('John', 'gold', None)
2022-11-28 22:24:58,468 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, status, address_id) VALUES (?, ?, ?)
2022-11-28 22:24:58,468 INFO sqlalchemy.engine.Engine [cached since 0.001685s ago] ('Jane', 'bronze', None)
2022-11-28 22:24:58,468 INFO sqlalchemy.engine.Engine COMMIT


Agreguemos una dirección a John

In [185]:
address = Address(street_name="Bogholder Allè", street_number=15, postnr=2720)
address

Bogholder Allè street_number=15 postnr=2720>

In [187]:
john.address = address

In [188]:
with Session(engine) as session:
    session.add(john)
    session.commit()

2022-11-28 22:26:41,396 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:26:41,397 INFO sqlalchemy.engine.Engine INSERT INTO addresses (street_name, street_number, postnr) VALUES (?, ?, ?)
2022-11-28 22:26:41,397 INFO sqlalchemy.engine.Engine [generated in 0.00024s] ('Bogholder Allè', 15, 2720)
2022-11-28 22:26:41,399 INFO sqlalchemy.engine.Engine SELECT customers.customer_id AS customers_customer_id, customers.name AS customers_name, customers.status AS customers_status 
FROM customers 
WHERE customers.customer_id = ?
2022-11-28 22:26:41,399 INFO sqlalchemy.engine.Engine [generated in 0.00032s] (1,)
2022-11-28 22:26:41,400 INFO sqlalchemy.engine.Engine UPDATE customers SET address_id=? WHERE customers.customer_id = ?
2022-11-28 22:26:41,400 INFO sqlalchemy.engine.Engine [generated in 0.00024s] (1, 1)
2022-11-28 22:26:41,400 INFO sqlalchemy.engine.Engine COMMIT


Ahora John va a comprar

In [189]:
potion = Purchase(item_name="Magic Potion", price=20.00, customer=john)
with Session(engine) as session:
    session.add(potion)
    session.commit()

2022-11-28 22:27:12,699 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:27:12,700 INFO sqlalchemy.engine.Engine SELECT customers.customer_id AS customers_customer_id, customers.name AS customers_name, customers.status AS customers_status, customers.address_id AS customers_address_id 
FROM customers 
WHERE customers.customer_id = ?
2022-11-28 22:27:12,700 INFO sqlalchemy.engine.Engine [generated in 0.00048s] (1,)
2022-11-28 22:27:12,704 INFO sqlalchemy.engine.Engine INSERT INTO purchases (item_name, price, user_id) VALUES (?, ?, ?)
2022-11-28 22:27:12,704 INFO sqlalchemy.engine.Engine [generated in 0.00101s] ('Magic Potion', 20.0, 1)
2022-11-28 22:27:12,705 INFO sqlalchemy.engine.Engine COMMIT


In [190]:
magic_hat = Purchase(item_name="Magic Hat", price=100)

In [191]:
with Session(engine) as session:
    # Necesitamos conectar a john a esta session
    session.add(john)
    # purchases es relación una-a-varios, entonces SA lo representa como lista
    john.purchases.append(magic_hat)
    session.add(john)
    session.commit()

2022-11-28 22:28:35,561 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:28:35,562 INFO sqlalchemy.engine.Engine SELECT customers.customer_id AS customers_customer_id, customers.name AS customers_name, customers.status AS customers_status, customers.address_id AS customers_address_id 
FROM customers 
WHERE customers.customer_id = ?
2022-11-28 22:28:35,562 INFO sqlalchemy.engine.Engine [generated in 0.00034s] (1,)
2022-11-28 22:28:35,564 INFO sqlalchemy.engine.Engine SELECT purchases.purchase_id AS purchases_purchase_id, purchases.item_name AS purchases_item_name, purchases.price AS purchases_price, purchases.user_id AS purchases_user_id 
FROM purchases 
WHERE ? = purchases.user_id
2022-11-28 22:28:35,564 INFO sqlalchemy.engine.Engine [generated in 0.00026s] (1,)
2022-11-28 22:28:35,565 INFO sqlalchemy.engine.Engine INSERT INTO purchases (item_name, price, user_id) VALUES (?, ?, ?)
2022-11-28 22:28:35,565 INFO sqlalchemy.engine.Engine [cached since 82.87s ago] ('Magic Hat', 

/var/folders/w6/2grt96qn39vgm51r4hz21bz40000gn/T/ipykernel_5527/675509969.py:5: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  john.purchases.append(magic_hat)


Se busca (select) igual que en Core

In [192]:
sql = sa.select(Customer).filter_by(name="Jane")
print(sql)


SELECT customers.customer_id, customers.name, customers.status, customers.address_id 
FROM customers 
WHERE customers.name = :name_1


In [193]:
with Session(engine) as session:
    jane = session.execute(sql).one_or_none()

2022-11-28 22:29:22,453 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:29:22,454 INFO sqlalchemy.engine.Engine SELECT customers.customer_id, customers.name, customers.status, customers.address_id 
FROM customers 
WHERE customers.name = ?
2022-11-28 22:29:22,455 INFO sqlalchemy.engine.Engine [generated in 0.00045s] ('Jane',)
2022-11-28 22:29:22,456 INFO sqlalchemy.engine.Engine ROLLBACK


In [195]:
jane

(Jane>,)

In [196]:
type(jane)

sqlalchemy.engine.row.Row

El resultado de esta query es un objeto `Row`, lo mismo que en Core, pero en modo ORM a veces nos interesamos en resultados escalares, el valor de la primera columna para cada fila

SA soporta esto por el modificador de escalares y los helpers de escalares

In [197]:
with Session(engine) as session:
    jane = session.execute(sql).scalars().one_or_none()

2022-11-28 22:31:44,438 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:31:44,439 INFO sqlalchemy.engine.Engine SELECT customers.customer_id, customers.name, customers.status, customers.address_id 
FROM customers 
WHERE customers.name = ?
2022-11-28 22:31:44,439 INFO sqlalchemy.engine.Engine [cached since 142s ago] ('Jane',)
2022-11-28 22:31:44,440 INFO sqlalchemy.engine.Engine ROLLBACK


In [198]:
jane

Jane>

In [199]:
type(jane)

__main__.Customer

In [200]:
with Session(engine) as session:
    jane = session.execute(sql).scalar_one_or_none()

2022-11-28 22:32:08,232 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:32:08,233 INFO sqlalchemy.engine.Engine SELECT customers.customer_id, customers.name, customers.status, customers.address_id 
FROM customers 
WHERE customers.name = ?
2022-11-28 22:32:08,233 INFO sqlalchemy.engine.Engine [cached since 165.8s ago] ('Jane',)
2022-11-28 22:32:08,235 INFO sqlalchemy.engine.Engine ROLLBACK


In [202]:
print (jane)
print (type(jane))

Jane>
<class '__main__.Customer'>


Si conocemos la clave primaria, SA provee un método eficiente para buscarla 

In [203]:
with Session(engine) as session:
    jane2 = session.get(Customer, jane.customer_id)

2022-11-28 22:33:07,447 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:33:07,449 INFO sqlalchemy.engine.Engine SELECT customers.customer_id AS customers_customer_id, customers.name AS customers_name, customers.status AS customers_status, customers.address_id AS customers_address_id 
FROM customers 
WHERE customers.customer_id = ?
2022-11-28 22:33:07,449 INFO sqlalchemy.engine.Engine [generated in 0.00046s] (2,)
2022-11-28 22:33:07,450 INFO sqlalchemy.engine.Engine ROLLBACK


In [204]:
jane2

Jane>

Ahora podemos preguntar por los atributos relacionados

In [205]:
with Session(engine) as session:
    session.add(john)
    print("Purchases:\t", john.purchases)
    print("Address:\t", john.address)

2022-11-28 22:33:44,273 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:33:44,275 INFO sqlalchemy.engine.Engine SELECT customers.customer_id AS customers_customer_id, customers.name AS customers_name, customers.status AS customers_status, customers.address_id AS customers_address_id 
FROM customers 
WHERE customers.customer_id = ?
2022-11-28 22:33:44,276 INFO sqlalchemy.engine.Engine [cached since 308.7s ago] (1,)
2022-11-28 22:33:44,277 INFO sqlalchemy.engine.Engine SELECT purchases.purchase_id AS purchases_purchase_id, purchases.item_name AS purchases_item_name, purchases.price AS purchases_price, purchases.user_id AS purchases_user_id 
FROM purchases 
WHERE ? = purchases.user_id
2022-11-28 22:33:44,278 INFO sqlalchemy.engine.Engine [cached since 308.7s ago] (1,)
Purchases:	 [Magic Potion>, Magic Hat>]
2022-11-28 22:33:44,279 INFO sqlalchemy.engine.Engine SELECT addresses.address_id AS addresses_address_id, addresses.street_name AS addresses_street_name, addresses.street

Si chequeamos un atributo no hay SQL emitido

In [207]:
john.status


<StatusEnum.gold: 'gold'>

#### 4.2 Carga de relaciones

Para acceder a los atributos de la relación, debemos estar dentro de una sesión, ya que, de forma predeterminada, las relaciones de SQLAlchemy son cargados de manera floja (*lazy-loading*).

Las consultas de *lazy-loading* generan consultas SQL adicionales cuando se accede a ellas para evitar cargar todos los datos relacionados en la memoria a la vez. La relación se puede configurar para que se cargue de [diferentes maneras](https://docs.sqlalchemy.org/en/14/orm/loading_relationships.html#relationship-loading-techniques), definidas en el constructor de relaciones o como opciones de `select`


In [208]:
from sqlalchemy.orm import joinedload, selectinload

with Session(engine) as session:
    sql = sa.select(Customer).options(joinedload(Customer.address), selectinload(Customer.purchases)).where(Customer.name == "John")
    john = session.execute(sql).unique().scalar_one()

2022-11-28 22:39:36,742 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:39:36,745 INFO sqlalchemy.engine.Engine SELECT customers.customer_id, customers.name, customers.status, customers.address_id, addresses_1.address_id AS address_id_1, addresses_1.street_name, addresses_1.street_number, addresses_1.postnr 
FROM customers LEFT OUTER JOIN addresses AS addresses_1 ON addresses_1.address_id = customers.address_id 
WHERE customers.name = ?
2022-11-28 22:39:36,745 INFO sqlalchemy.engine.Engine [generated in 0.00050s] ('John',)
2022-11-28 22:39:36,747 INFO sqlalchemy.engine.Engine SELECT purchases.user_id AS purchases_user_id, purchases.purchase_id AS purchases_purchase_id, purchases.item_name AS purchases_item_name, purchases.price AS purchases_price 
FROM purchases 
WHERE purchases.user_id IN (?)
2022-11-28 22:39:36,747 INFO sqlalchemy.engine.Engine [generated in 0.00055s] (1,)
2022-11-28 22:39:36,748 INFO sqlalchemy.engine.Engine ROLLBACK


In [209]:
john

John>

Alternativamente, podemos definir la relación para que no sea `lazy`: agreguemos una tabla de `loyalty_points` que registre cuántos puntos de fidelidad tiene una compra determinada.

In [210]:
@mapper_registry.mapped
class LoyaltyPoints:
    __tablename__ = "loyalty_points"
    __table_args__ = {"extend_existing": True}
    
    loyalty_point_id: int = sa.Column(sa.Integer, primary_key=True)
    customer_id: int = sa.Column(sa.Integer, sa.ForeignKey("customers.customer_id"))
    purchase_id: int = sa.Column(sa.Integer, sa.ForeignKey("purchases.purchase_id"))
    total_points: int = sa.Column(sa.Integer)
    
    # One-to-one relationship
    purchase: Purchase = relationship("Purchase", backref="points", lazy="joined")
    
    # One-to-one
    customer: Customer = relationship("Customer", backref="points", lazy="selectin")

In [211]:

mapper_registry.metadata.create_all(engine)

2022-11-28 22:41:54,006 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:41:54,007 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2022-11-28 22:41:54,008 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 22:41:54,009 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("purchases")
2022-11-28 22:41:54,009 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 22:41:54,009 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customers")
2022-11-28 22:41:54,009 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 22:41:54,010 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("loyalty_points")
2022-11-28 22:41:54,010 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 22:41:54,010 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("loyalty_points")
2022-11-28 22:41:54,010 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 22:41:54,011 INFO sqlalchemy.engine.Engine 
CREATE TABLE loyalty_points (
	loyalty_point_id INTEGER NOT NULL, 
	customer_id

Agreguemos unos loyalty points

In [212]:
with Session(engine) as session:
    sql = sa.select(Customer).filter_by(name="John")
    john = session.execute(sql).scalar_one()
    loyalty_purchase = john.purchases[0]
    loyalty_points = LoyaltyPoints(customer=john, purchase=loyalty_purchase, total_points=1000)
    session.add(loyalty_points)
    session.commit()

2022-11-28 22:42:21,122 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:42:21,127 INFO sqlalchemy.engine.Engine SELECT customers.customer_id, customers.name, customers.status, customers.address_id 
FROM customers 
WHERE customers.name = ?
2022-11-28 22:42:21,127 INFO sqlalchemy.engine.Engine [generated in 0.00052s] ('John',)
2022-11-28 22:42:21,128 INFO sqlalchemy.engine.Engine SELECT purchases.purchase_id AS purchases_purchase_id, purchases.item_name AS purchases_item_name, purchases.price AS purchases_price, purchases.user_id AS purchases_user_id 
FROM purchases 
WHERE ? = purchases.user_id
2022-11-28 22:42:21,129 INFO sqlalchemy.engine.Engine [cached since 825.6s ago] (1,)
2022-11-28 22:42:21,130 INFO sqlalchemy.engine.Engine INSERT INTO loyalty_points (customer_id, purchase_id, total_points) VALUES (?, ?, ?)
2022-11-28 22:42:21,131 INFO sqlalchemy.engine.Engine [generated in 0.00038s] (1, 1, 1000)
2022-11-28 22:42:21,131 INFO sqlalchemy.engine.Engine COMMIT


Veamos que pasa si seleccionamos a john nuevamente

In [213]:
with Session(engine) as session:
    sql = sa.select(LoyaltyPoints).where(LoyaltyPoints.customer.has(Customer.name == "John"))
    john_points = session.execute(sql).scalar_one()

2022-11-28 22:43:02,010 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 22:43:02,012 INFO sqlalchemy.engine.Engine SELECT loyalty_points.loyalty_point_id, loyalty_points.customer_id, loyalty_points.purchase_id, loyalty_points.total_points, purchases_1.purchase_id AS purchase_id_1, purchases_1.item_name, purchases_1.price, purchases_1.user_id 
FROM loyalty_points LEFT OUTER JOIN purchases AS purchases_1 ON purchases_1.purchase_id = loyalty_points.purchase_id 
WHERE EXISTS (SELECT 1 
FROM customers 
WHERE customers.customer_id = loyalty_points.customer_id AND customers.name = ?)
2022-11-28 22:43:02,013 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ('John',)
2022-11-28 22:43:02,014 INFO sqlalchemy.engine.Engine SELECT customers.customer_id AS customers_customer_id, customers.name AS customers_name, customers.status AS customers_status, customers.address_id AS customers_address_id 
FROM customers 
WHERE customers.customer_id IN (?)
2022-11-28 22:43:02,015 INFO sqlalchemy.

In [214]:
john_points.purchase

Magic Potion>

La carga de relaciones es uno de los mayores beneficios de los ORM, pero también puede ser la forma más fácil de meter las patas. Hay que tener ojo en la estrategia de relaciones